![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [2]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

ModuleNotFoundError: No module named 'pandas'

In [7]:
# Start coding here
# Use as many cells as you need

## Which hour has the highest frequency of crimes? Store as an integer variable called peak_crime_hour.

In [8]:
# Get the number of unique DR_NO values

num_unique_DR_NO = crimes['DR_NO'].nunique()

num_unique_DR_NO

247988

In [9]:
# Getting number of unique crimes by TIME OCC and sorting descending

crime_hours = crimes.groupby(by='TIME OCC')['DR_NO'].nunique().sort_values(ascending=False)
crime_hours

TIME OCC
1200    8333
1800    6183
1700    5824
2000    5606
1900    5250
        ... 
2259       5
2254       5
0756       5
0542       5
0531       3
Name: DR_NO, Length: 1439, dtype: int64

In [10]:
# Casting peak_crime_hour string to integer

peak_crime_hour = int(str(crime_hours.index[0])[:2])

# Result: Peak crime hour is 1200
peak_crime_hour

12

## Which area has the largest frequency of night crimes (crimes committed between 10pm and 4am)? Save as a string variable called peak_night_crime_location.

In [11]:
# Casting TIME OCC as int

crimes['TIME OCC'] = crimes['TIME OCC'].astype(int)
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,221412410,2022-06-15,2020-11-12,1700,Pacific,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),0,NaN,NaN,NaN,Invest Cont,13600 MARINA POINT DR
1,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
2,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
3,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
4,231207725,2023-02-27,2020-01-27,635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV


In [12]:
# Getting subset of crimes df called night crimes (between 2200 and 0400) 

# Define condition of night crimes

nc_condition = (crimes['TIME OCC'] >= 2200) | (crimes['TIME OCC'] <= 400)

# Subset original df according to condition
night_crimes = crimes[nc_condition]
night_crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
9,231207476,2023-02-27,2020-08-15,1,77th Street,BURGLARY,72,M,B,NaN,Invest Cont,8800 HAAS AV
12,221711184,2022-06-15,2020-05-15,155,Devonshire,THEFT OF IDENTITY,27,M,B,NaN,Invest Cont,8300 WHITE OAK AV
36,221314362,2022-07-11,2020-04-07,1,Newton,THEFT OF IDENTITY,53,F,H,NaN,Invest Cont,1600 E OLYMPIC BL
39,231307252,2023-03-03,2020-07-05,2305,Newton,THEFT OF IDENTITY,22,F,B,NaN,Invest Cont,6600 S BROADWAY
42,221614254,2022-11-13,2020-01-01,1,Foothill,THEFT OF IDENTITY,22,F,H,NaN,Invest Cont,10200 TELFAIR AV


In [13]:
# Getting the list of areas with count of night crimes sorted descending

night_crimes_areas = crimes.groupby(by='AREA NAME')['DR_NO'].nunique().sort_values(ascending=False)
night_crimes_areas

AREA NAME
Central        18732
77th Street    15408
Southwest      14194
Pacific        13866
Olympic        12792
Newton         12790
Hollywood      12761
Southeast      12600
Wilshire       12080
N Hollywood    11728
Rampart        11725
West LA        11169
West Valley    10766
Northeast      10686
Van Nuys       10640
Devonshire     10184
Topanga         9968
Harbor          9701
Mission         9175
Hollenbeck      9063
Foothill        7960
Name: DR_NO, dtype: int64

In [14]:
# Result: Peak night crime location is '10200    SANTA MONICA                 BL'

peak_night_crime_location = str(night_crimes_areas.index[0])
peak_night_crime_location

'Central'

## Identify the number of crimes committed against victims by age group (<18, 18-25, 26-34, 35-44, 45-54, 55-64, 65+). Save as a pandas Series called victim_ages.

In [15]:
age_bins = [0, 17, 25, 34, 44, 54, 64, np.inf]
age_labels = ["<18", "18-25", "26-34", "35-44", "45-54", "55-64", "65+"]

# Add a new column using pd.cut() to bin values into discrete intervals
crimes["Age Bracket"] = pd.cut(crimes["Vict Age"],
                               bins=age_bins,
                               labels=age_labels)

# Find the category with the largest frequency
victim_ages = crimes["Age Bracket"].value_counts()
victim_ages

26-34    47470
35-44    42157
45-54    28353
18-25    28291
55-64    20169
65+      14747
<18       4528
Name: Age Bracket, dtype: int64